# Extrahieren der Textstufen

In [2]:
import lxml
from lxml import etree
import pandas as pd

In [3]:
tree = etree.parse("C:/Users/jO/Desktop/DH_MA_Arbeit/coding/modelliert_output_pos_vwdazb_trf.xml")

In [4]:
ns = {"t":'http://www.tei-c.org/ns/1.0'}
xml = "{http://www.w3.org/XML/1998/namespace}"

## Versuche, die richtigen XPath Statements zu formulieren - funktionieren auch!

### Extrahierung vom "originalen" Typoskript: 
- <add ignorieren
- <reg ignorieren
- <del einbeziehen
- <orig einbeziehen

In [5]:
wit1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']", namespaces=ns)
linegroups = tree.xpath("//t:lg", namespaces=ns)
lines = tree.xpath("//t:l", namespaces=ns)

for linegroups in wit1: 
    for lines in linegroups: 
        wit1typ = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']//t:l/text() | //t:div[@type='witness' and @xml:id='nr.1']//t:del/text() | //t:div[@type='witness' and @xml:id='nr.1']//t:orig/text()", namespaces=ns)
        wit1typ = "".join(wit1typ)
        wit1typ.replace("\n", "").replace("\r", "")
print("Originaltyposkript:", wit1typ)

Originaltyposkript: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT die_/DET ihre_/DET 
                            Gedanken_/NOUN durch_/ADP die_/DET Feder_/NOUN 
                            sichtbar_/ADV zu_/PART mac

### Extrahierung von @change-Values Versionen: 
- <add  von @change="_pen" <hi>einbeziehen</hi>
- <reg von @change="_pen" einbeziehen
- <del einbeziehen, wenn es nicht @change="_pen" ist 
- <orig einbeziehen, wenn <reg nicht von @change="_pen" ist

In [6]:
# alle Versionen nach den verschiedenen @change-Values:
# mal nur aus dem ersten Textträger <div type='witness' @xml:id='nr.1'>

divwit1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']//t:l/descendant-or-self::*", namespaces=ns)
allattributevalues = set()

for child in divwit1:
    attribute_value = child.attrib.get('change')
    if attribute_value:
        if attribute_value not in allattributevalues:
            allattributevalues.add(attribute_value)
print(f"Unterschiedliche Revisionen: {allattributevalues}\n")
            
        
for attributevalue in allattributevalues:
    textname = f"{attributevalue}text"
    text = tree.xpath(f"""//t:div[@type='witness' and @xml:id='nr.1']//t:l/text() | 
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:add[@change='{attributevalue}']/text() | 
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:del[not(@change='{attributevalue}')]/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:reg[@change='{attributevalue}']/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)

    textname = "".join(text)
    textname.replace("\n", "").replace("\r", "")
    print(f"Typoskript mit Revisionen nach {attributevalue}:\n {textname} \n \n")


Unterschiedliche Revisionen: {'#black_pen', '#typewriter'}

Typoskript mit Revisionen nach #black_pen:
 der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET Brust_/NOUN ,_/PUNCT 
                            mit_/ADP der_/DET linken_/ADJ Hand_/NOUN 
                            3_/NUM mal_/ADV geteilt_/VERB 
                             (_/PUNCT die_/DET 
                            die_/DET Rosalia_/PROPN sichtbar_/ADV zu_/PART machen_/VERB ,_/PUNCT )_/PUNCT bewaldet_/ADV ,_/PUNCT wir_/PRON essen_/VERB 
                            kleine_/ADJ Drachen_/NOUN ,_/PUNCT den_/DET Mond_/NOUN ,_/PUNCT 
                

## Versuch: Extrahierung aller Textstufen according to Vorkommen von @change-values in allen <div type="witness" 

### Versuch, daraus eine Funktion zu machen, die alle möglichen Textstufen extrahiert

#### Mit extra vielen print(), damit jeder Vorgang nachvollzogen werden kann

In [25]:
# funktioniert prinzipiell -  Versionen werden aber mehrmals eingefügt (ca. 4mal)

processed_divs = set() #speichert den Namen aller divs, die durch die processalldivs()-Funktion gehen
processed_texts = set() # speichert den Namen alles texte, die durch die addtext()-Funktion gehen

#Funktion, die den extrahierten xpath-text weiterverarbeitet, ein <l>-tag erstellt und den text im neuen tag speichert (in der Hierarchie <div type="witness"><div><lg><l>text ...)
# meine Vermutung, dass es innerhalb dieser Funktion evtl mehrmals geadded wird
def addtext(text, poemdiv, textname): 
    if textname not in processed_texts:    
        text = "".join(text)
        text = text.replace("\n", "").replace("\r", "")
        text = " ".join(text.strip().split())
        print(f"\n Neuer extrahierter Text:\n > Name: {textname}: \n > Text: {text}")
        print(f"\n Der Text von '{textname}' wird in xml geladen ...")
        linetag = etree.Element("l")
        print("1. linetag wurde erstellt")
        linetag.text = text # den extrahierten text als Inhalt des neuen divs definieren 
        print("2. text wurde dem linetag hinzugefügt")
        linegrouptag = etree.Element("lg") 
        print("3. linegrouptag wurde erstellt")
        linegrouptag.append(linetag)
        print("4. linetag wurde dem linegrouptag hinzugefügt")
        divtag = etree.Element("div")
        divtag.set("type", f"{textname}")  # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen      
        print("5. divtag wurde erstellt")
        divtag.append(linegrouptag)
        print("6. linegrouptag wurde zu divtag hinzugefügt")
        poemdiv.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen
        print("7. divtag wurde dem div hinzugefügt")  
        print(f" ... und '{textname}' wurde zum xml hinzugefügt!\n")
        processed_texts.add(textname)
        if textname in processed_texts: 
            print(f"verarbeitete texte sind: {processed_texts}")
            return
    else:
        print(f"{textname} wurde schon verarbeitet!")
        return    
        
def processalldivs(divs, poemdiv):  
    divlen = len(divs)
    counter = 0
    print(f"Es sind {len(divs)} divs im XML-Dokument")
    if counter <= divlen:
        for div in divs: 
            divtype = div.attrib.get('type')
            divid = div.attrib.get(f'{xml}id')
            divname = f"{divtype}_{divid}"
            print(f"! Für folgendes <div> werden alle @change-Versionen extrahiert: '{divname}' !")
            if divname not in processed_divs:
                #"original"-Version herausnehmen (=heißt: den Text in <l>, <del> und <orig> beachten):
                textname = f"{divname}_original"
                text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
                addtext(text, poemdiv, textname) #Funktion zur Verarbeitung und Vpeicherung des extrahierten Textes
            
                allattributevalues = set() #set für alle @change-attributevalues pro div   
                
                # der Text mit den Revisions-Tags befindet sich jeweils im Linetag <l>:
                lines = tree.xpath(f"//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/descendant-or-self::*", namespaces=ns)

                # für die Line im <div> alle attributevalues herausholen, um dafür später eigene Revisions-Textstufen zu extrahieren
                for line in lines:
                    attribute_value = line.attrib.get('change') #unterschiedliche values von @change in allen tags innerhalb einer line abrufen
                    if attribute_value: 
                        if attribute_value not in allattributevalues:
                            allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
                print(f"\n Es gibt folgende Revisionen im Text '{divname}': {allattributevalues}\n") # alle gefundenen @change-values printen 

                # pro @change-value, das im set gespeichert wurde, wird der spezifische Revisionstext abgerufen:
                for attributevalue in allattributevalues: 
                    textname = f"{divname}_{attributevalue}"
                    # pro Change Text beachten in <l>, <add>, <reg> aber nicht in <del> & <orig> (wenn reg dasselbe value hat wie <orig>)
                    text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
                    addtext(text, poemdiv, textname) #Funktion zur Verarbeitung und Speicherung des Textes
                processed_divs.add(divname)
                counter += 1
                print(f"\n!!! {counter} div/s wurden verarbeitet!\n")
            else:
                print("All done")
                return
        if counter == divlen:
            print("All done")
            return


In [26]:
#Funktion abrufen
poemdiv = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']", namespaces=ns)
divs = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']", namespaces=ns)
processalldivs(divs, poemdiv)
tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)

Es sind 7 divs im XML-Dokument
! Für folgendes <div> werden alle @change-Versionen extrahiert: 'witness_nr.1' !

 Neuer extrahierter Text:
 > Name: witness_nr.1_original: 
 > Text: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV schließen_/VERB ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN in_/ADP der_/DET Westentasche_/NOUN ,_/PUNCT oder_/CCONJ als_/SCONJ müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON nötig_/ADV haben_/VERB ,_/PUNCT die_/DET ihre_/DET Gedanken_/NOUN durch_/ADP die_/DET Feder_/NOUN sichtbar_/ADV zu_/PART machen_/VERB ,_/PUNCT )_/PUNCT die_/DET Rosalia_/PROPN bewaldet_/ADV ,_/PUNCT wir_/PRON essen_/VERB kleine_/ADJ Drachen_/NOUN ,_/PUNCT den_/DET Mond_/NOUN ,_/PUNCT Asche_/NOUN sein_/AUX ,_/PUNCT das_/DET Land_/NOUN ist